#### Kapitel 4
## **NLP-Ansatz**
---

### Natural Language Processing
- Verarbeitung natürlicher Sprache
- nutzt verschiedene Verfahren der Textverarbeitung
    - Tokenisierung
    - Lemmatisierung
    - Stoppwortentfernung
- Kombination dieser Verfahren auch als *Vorverarbeitung (preprocessing)* bezeichnet

### Vorverarbeitung mit **SpaCy**
- entwickelt von Explosion AI
- quelloffene Python-Bibliothek für NLP
- zahlreiche Sprachmodelle variabler Größe in verschiedenen Sprachen
- unterstützt alle wichtigen Textverarbeitungsverfahren
- Zusammenfassung einzelner Schritte als Pipeline
- bringt eine Reihe vorgefertigter Pipelines mit

### Training mit **DistilBERT**
- **BERT** (*Bidirectional Encoder Representations from Transformers*)
    - ist ein quelloffenes NLP-Sprachmodell von Google
    - basiert auf der Transformer-Architektur
    - Besonderheit: arbeitet bidirektional
    - analysiert Text in beide Richtungen
- **DistilBERT**
    - "destillierte" Version von BERT
    - 40 % kleiner
    - 60 % schneller
    - 97 % der Qualität

___

# Natural Language Processing angewandt auf Welfake
___

In [2]:
pip install torch --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install spacy && python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 4.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.6/12.8 MB 4.6 MB/s eta 0:00:03
     --------- ------------------------------ 2.9/12.8 MB 4.8 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 4.4 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 4.3 MB/s eta 0:00:02
     ----------------- ---------------------- 5.5/12.8 MB 4.6 MB/s eta 0:00:02
     --------------------- ------------------ 6.8/12.8 MB 4.9 MB/s eta 0:00:02
     ------------------------- -------------- 8.1/12.8 MB 5.1 MB/s eta 0:00:01
     ------------------------------ --------- 9.7/12.8 MB 5.3 MB/s eta 0:00:01
     ---------------------------------- ----- 11.0/12.8 MB 5.5 MB/s eta 0:00:01
     -------------------------------------- - 12.3/12.8 MB 5.5 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 M

In [4]:
# Imports
import pandas as pd
import spacy
import torch
import transformers
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from datasets import Dataset
from pandarallel import pandarallel

In [5]:
# Version Checks
print("Pandas version:", pd.__version__)
print("SpaCy version:", spacy.__version__)
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)
print("Cuda version:", torch.version.cuda)

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# Initialize Parallelization
pandarallel.initialize(progress_bar=True)

Pandas version: 2.3.0
SpaCy version: 3.8.7
Transformers version: 4.52.4
Torch version: 2.5.1+cu121
Cuda version: 12.1
GPU: NVIDIA GeForce RTX 2060 SUPER
INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [15]:
# Data Path
csv_path = "../src/data/Saurabh Shahane - Fake_News_Classification/WELFake_Dataset.csv"

# Read with semicolon separator
df = pd.read_csv(csv_path, sep=',')

# Split Data
df_train, df_temp = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['label'])

# Quick sanity check
print("Train:", df_train.shape)
print("Eval: ", df_val.shape)
print("Test: ", df_test.shape)

# first 5 rows of dataset
with pd.option_context('display.max_colwidth', None):
    print(df_train.head(5))

Train: (50493, 4)
Eval:  (10820, 4)
Test:  (10821, 4)
       Unnamed: 0  \
24388       24388   
40608       40608   
66652       66652   
71224       71224   
17060       17060   

                                                                                                                                  title  \
24388                              THE FACE OF THE DEMOCRAT PARTY Has A Message For The Tea Party And You Won’t Want To Miss It…[VIDEO]   
40608  INCOMING FRESHMEN Are Put On Notice With Welcome Letter From U of Chicago Dean Of Students…”Trigger Warning” Crybabies Stay Home   
66652                                     REPUBLICANS CALL FOR ANSWERS: Did Wasserman-Schultz and Podesta Just Get Caught In A Big Lie?   
71224                         EXTORTION? HOW IRAN Used Nuke Deal To Force Obama To Retreat From Embarrassing “Red Line” Threat To Syria   
17060                                                                   Democrats want strong response to intel report on 20

### Preprocessing Pipeline
- Lower Case
    - Kleinschreibung
- Tokenisierung
    - Aufteilung des Textes in kleinste Einheiten ("tokens")
- Lemmatisierung
    - gebeugte Wörter werden auf ihren Stamm ("lemma") zurückgeführt
    - Bsp.: *going* → *go*
- Stop Word Removal
    - Entfernung häufiger Wörter mit größtenteils grammatikalischer Funktion und wenig Inhalt ("stop words")
    - Bsp.: *the*, *and*, *in*, *of*, *with*, *but* etc.
- Punctuation Removal
    - Entfernung von Satzzeichen

In [21]:
# Define Preprocessing
def preprocess_text(text, print_tokens=False):
    import spacy
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

    if print_tokens:
        print(tokens)

    return " ".join(tokens)

In [22]:
# Test Preprocessing (can be skipped)
df_train_sample = df_train[:5]['text'].astype(str).apply(preprocess_text, print_tokens=True)

['ass', 'clown', 'remind', 'term', 'limit', 'important', 'leisa', 'see', 'rep.', 'rangel', 'd', 'ny', 'close', 'personal', 'week', 'visit', 'capitol', 'building', 'guest', 'rep.', 'mike', 'bishop', 'r', 'mi', 'shock', 'people', 'elect', 'represent', 'nation', '20', 'house', 'member', 'look', 'like', 'roam', 'hall', 'nursing', 'home', ' ', 'hall', 'congress', ' ', 'people', 'business', 'make', 'decision', 'behalf', 'country', 'charlie', 'rangel', 'perfect', 'example', 'assertion', 's', 'secret', 'democrats', 't', 'stand', 'tea', 'party', 'rarely', 'express', 'hatred', 'loud', 'like', 'rep.', 'charlie', 'rangel', 'moment', 'catch', 'camera', 'democrat', 'congressman', 'let', 'reporter', 'know', 'think', 'coalition', 'citizen', 'believe', 'individual', 'liberty', 'small', 'government', 'undoubtedly', 'constituents).theblaze', 'report', 'rep.', 'charlie', 'rangel', 'd', 'n.y.', 'harsh', 'word', 'tea', 'party', 'republicans', 'town', 'hall', 'host', 'tuesday', 'new', 'york', 'city', 'outspo

In [23]:
# Apply Preprocessing
df_train['text'] = df_train['text'].astype(str).parallel_apply(preprocess_text)
df_test['text'] = df_test['text'].astype(str).parallel_apply(preprocess_text)

print(df_train['text'].head(5))

24388    ass clown remind term limit important leisa se...
40608    wow university chicago send impressive letter ...
66652    bust moderate republican susan collins maine s...
71224    european force deal massive influx muslim refu...
17060    washington reuters democrats u.s. senate house...
Name: text, dtype: object


In [24]:
# Load Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenizing
train_encodings = tokenizer(list(df_train['text']), truncation=True, padding=True)
test_encodings = tokenizer(list(df_test['text']), truncation=True, padding=True)

### Die wichtigsten Trainingsparameter
- `num_train_epochs=2`
    - Anzahl der Trainingsepochen
    - ausreichend für großen, diversen Trainingsdatensatz
    - zu wenige Epochen = zu allgemeines Modell (*underfitting*)
    - zu viele Epochen = zu spezielles Modell (*overfitting*)
- `per_device_train_batch_size=32`
    - Schrittgröße beim Training
    - kleinere Batches erzeugen Rauschen
    - Rauschen verhindert Overfitting
- `weight_decay=0.01`
    - Form der L2-Regularisierung
    - verhindert Overfitting durch Bestrafung zu hoher Gewichte

In [25]:
# Translate to Huggingface Dataset Format
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'label': list(df_train['label'])
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'label': list(df_test['label'])
})

# Load Model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Training Parameters
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Define Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    report = classification_report(labels, preds, output_dict=True)
    return {
        "accuracy": report["accuracy"],
        "f1": report["weighted avg"]["f1-score"]
    }

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Start Training
trainer.train()

# Evaluation
predictions = trainer.predict(test_dataset)
pred_labels = torch.argmax(torch.tensor(predictions.predictions), dim=1)
print(classification_report(df_test['label'], pred_labels))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.048800,0.045197,0.984659,0.984661
2,0.013800,0.043768,0.988079,0.988079


              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5255
           1       0.99      0.99      0.99      5566

    accuracy                           0.99     10821
   macro avg       0.99      0.99      0.99     10821
weighted avg       0.99      0.99      0.99     10821



- - -
- - -
- - -

#### Kapitel 8
## Ausblick

### Mehrklassige Datensätze
Die meisten Fake News Datensätze sind binär klassifiziert.  
Es gibt aber auch Datensätze, die deutlich komplexer aufgebaut sind.
___
#### Beispiel 1: **[LIAR](https://datasets.activeloop.ai/docs/ml/datasets/liar-dataset/)**
*LIAR* enthält 12,8 Tausend Phrasen und kurze Statements aus einem Zeitraum von 10 Jahren, welche in 6 Kategorien unterteilt sind.  
Die Kategorien sind dabei nach Glaubwürdigkeit sortiert, erweitern den binären Klassifikator also um eine simple Abstufung.
- `true`
- `mostly-true`
- `half-true`
- `barely-true`
- `false`
- `pants-fire`
___
#### Beispiel 2: **[Fakeddit](https://github.com/entitize/Fakeddit)**
*Fakeddit* ist eine Initiative zur Klassifizierung von Newsartikeln.  
Über eine CodaLab Competition wurden im Zeitraum von 2020 bis 2022 über 1 Million Artikel klassifiziert.  
Dabei waren folgende 6 Klassen vorgegeben.
- `true`
- `satire/parody`
- `misleading content`
- `manipulated content`
- `false connection`
- `imposter content`
___
#### Beispiel 3: **[andyP](https://huggingface.co/datasets/andyP/fake_news_en_opensources)**
Dieser Datensatz enthält 5,9 Millionen Einträge, welche in 12 Klassen eingeteilt sind.
- `reliable`
- `ploitical`
- `bias`
- `fake`
- `conspiracy`
- `rumor`
- `unknown`
- `clickbait`
- `unreliable`
- `satire`
- `junksci`
- `hate`
___
Komplexere Einteilungen schaffen ein deutlich relistischeres Bild der Online-Newsartikellandschaft, benötigen jedoch einen ausreichend große und qualitativ hochwertige Datensätze.